# Fluvial corridor workflow

In [ ]:
## Install GDAL
For Windows 10, install GDAL with OSGEO4W

[link to installer](https://trac.osgeo.org/osgeo4w/wiki/OSGeo4W_fr)

Choose *Express Install* then GDAL.

In your user environement variables add C:\OSGeo4W\bin to PATH.

## Set input parameters
Create a config.ini file.

Create a .env text file in 01-drainage_plan with
FCT_CONFIG=./config.ini

The outputs folder and name are generate  automatically.
They are define in yml files in the config/datasets folder or by tiledir attributes in config.ini section.

Activate the virtual environement in shell from the python-fct folder.

The initial DEM (RGEALTI) is splited into tiles with 10K_TILESET and 10KBIS_TILESET with this shell commands.

fct-tiles call shell FileCommand.py and TileCommand.py
Arguments :
- config.ini = path to config.ini with input path and parameters
- extract = Extract Tiles from Datasource for tiles defined in Tileset, and store as Dataset. Function TileCommand.py/extract/Tiles.py/DatasourceToTiles
- bdalti = input define in config.ini
- 10k = input tile grid define in config.ini
- dem = output file define in config/datasets/drainage.yml

In [1]:
! fct-tiles -c ./config.ini extract bdalti 10k dem
! fct-tiles -c ./config.ini extract bdalti 10kbis dem

FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env

FCT_CONFIG=./config.ini
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env

FCT_CONFIG=./config.ini


Build virtual raster from the tiled previously created.

Arguments :
- config.ini = path to config.ini with input path and parameters
- buildvrt = Build GDAL Virtual Raster from tile dataset. Function TileCommand.py/vrt/tileio.py/buildvrt
- 10k = input tile grid define in config.ini
- dem = output file define in config/datasets/drainage.yml

In [2]:
! fct-tiles -c ./config.ini buildvrt 10k dem
! fct-tiles -c ./config.ini buildvrt 10kbis dem

FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.


If you have two different scales DEM, you can fill the precise one with the less precise
A faire en amont du lissage?

## Create a drainage networks
### DEM preparation
#### create configuration file

In [3]:
from fct.drainage import PrepareDEM
# the confi.ini file is load from the .env text file

FCT_CONFIG=./config.ini


PrepareDEM.config gather all the input and output parameters
input are defined in config.ini
output are define in .yml files in config/datasets
PrepareDEM.config have 3 dictionnaries
- _tilesets for raster tiles
- _datasources for input files
- _datasets for output destination files with parameters

In [16]:
# uncomment below to see configuration definition and function
# help(PrepareDEM.config)

In [19]:
# uncomment code below to print all parameters from the config class
# vars(PrepareDEM.config)
# example : get all data sources from the config class
# PrepareDEM.config._datasources

#### DEM smoothing

In [4]:
# Prepare the smoothing parameter
params = PrepareDEM.SmoothingParameters()

smoothing parameters : 
- elevations = input
- output = output smoothed dem
- window = smoothing window

In [6]:
# uncomment below to print default smoothing parameter

In [7]:
# uncoment below to get input path 
# params.elevations.filename()

In [5]:
# change window size
params.window=2

In [9]:
# uncomment below to print new parameters
# params.__dict__.items()

In [6]:
# Mean filter for each tile of the tileset
PrepareDEM.MeanFilter(params, overwrite=True, processes=4, tileset='10k')
PrepareDEM.MeanFilter(params, overwrite=True, processes=4, tileset='10kbis')

In [12]:
# uncomment below to see an example of the the output tile path
# params.output.tilename(row=1, col=1)

Create virtual raster from tiles in shell (optional)

In [7]:
! fct-tiles -c ./config.ini buildvrt 10k smoothed
! fct-tiles -c ./config.ini buildvrt 10kbis smoothed

FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.


#### Fill sink

In [12]:
# import module
from fct.drainage import DepressionFill

In [13]:
# set parameters
params = DepressionFill.Parameters()

DepressionFill parameters :

- _elevations = input
- _hydrography = Stream network derived from cartography draped on DEM **a quoi ça sert?? c'est dans la fonction BurnTile pour creuser le MNT (avec le paramètre offset) avec un réseau hydrographique si cette couche est disponible (sauf que l'emplacement du fichier shp à fournir dans le yml est dans les outputs et non dans les input). J'ai l'impression que ce n'est plus utilisé...**
- _filled = output filled DEM tiles define in yml. DEM tiles, depression filling procedure, first pass: individual tile processing
- _labels = output watershed label raster tiles define in yml. Watershed labels (depression filling procedure)
- _graph = output watershed graph define in yml. Cross-tile watershed connection graph (depression filling procedure)
- _spillover = output watershed spillover define in yml. Tile spillovers, z resolution of dem-watershed-graph (depression filling procedure)
- _resolved = output final filled DEM tiles define in yml. DEM tiles, depression filling procedure, second pass: cross-tile resolution based on spillover graph
- _offset = ? **Est utiliser par la fonction BurnTile pour modifier les donnes du MNT avec le réseau hydrographique, par défault la valeur est de -1, cependant si elle est égale à -1 le réseau est surélevé et non creusé...**
- _exterior_data = DEM edge value. Default 9000 to create wall to avoid flow to get out.

In [21]:
# uncomment below to show default parameters
# params.__dict__.items()

In [15]:
# change default parameters
params.elevations = 'smoothed'
params.exterior_data = 0.0

In [17]:
# Identifies and numbers the watersheds and continuous areas of the same elevation, with filling in the holes
DepressionFill.LabelWatersheds(params, overwrite=True, processes=4, tileset='10k')
DepressionFill.LabelWatersheds(params, overwrite=True, processes=4, tileset='10kbis')

# Calculates the overflow graph between the different tiles
DepressionFill.ResolveWatershedSpillover(params, overwrite=True, tileset='10k')
DepressionFill.ResolveWatershedSpillover(params, overwrite=True, tileset='10kbis')

# Adjusts the elevation of tile edge depressions, (altitude differential with the overflow point)
DepressionFill.DispatchWatershedMinimumZ(params, overwrite=True, processes=4, tileset='10k')
DepressionFill.DispatchWatershedMinimumZ(params, overwrite=True, processes=4, tileset='10kbis')



Build spillover graph

Resolve Watershed's Minimum Z
Saved to : ..\outputs\GLOBAL\DEM\WATERSHED_SPILLOVER_10K.npz
Build spillover graph

Resolve Watershed's Minimum Z
Saved to : ..\outputs\GLOBAL\DEM\WATERSHED_SPILLOVER_10KBIS.npz




Create virtual raster from tiles in shell (optional)

In [18]:
! fct-tiles -c ./config.ini buildvrt 10k dem-watershed-labels
! fct-tiles -c ./config.ini buildvrt 10k dem-filled
! fct-tiles -c ./config.ini buildvrt 10k dem-filled-resolved

! fct-tiles -c ./config.ini buildvrt 10kbis dem-watershed-labels
! fct-tiles -c ./config.ini buildvrt 10kbis dem-filled
! fct-tiles -c ./config.ini buildvrt 10kbis dem-filled-resolved

FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorial

#### Resolve flats

In [19]:
# import module
from fct.drainage import BorderFlats

In [20]:
# set parameters
params = BorderFlats.Parameters()

BorderFlats parameters :

- _labels = watersheds with labels create in fill sinks step.
- _resolved = filled DEM tiles create in fill sinks, depression filling procedure, second pass: cross-tile resolution based on spillover graph
- _flat_labels = Flat labels output raster (depression filling procedure)
- _flat_graph = Cross-tile flat output connection graph (depression filling procedure)
- _flat_spillover = Tile spillovers output, z resolution of dem-flat-graph (depression filling procedure)
- _output = BorderFlats output. Filled DEM tiles, depression filling procedure, second pass: cross-tile resolution based on spillover graph

In [19]:
# uncomment below to show default parameters
# params.__dict__.items()

In [22]:
# indentify flat area in previous filled DEM tiles and fix flat border
BorderFlats.LabelBorderFlats(params=params, processes=4, tileset='10k') 
BorderFlats.LabelBorderFlats(params=params, processes=4, tileset='10kbis')

BorderFlats.ResolveFlatSpillover(params=params, tileset='10k')
BorderFlats.ResolveFlatSpillover(params=params, tileset='10kbis')

BorderFlats.DispatchFlatMinimumZ(params=params, overwrite=True, processes=4, tileset='10k')
BorderFlats.DispatchFlatMinimumZ(params=params, overwrite=True, processes=4, tileset='10kbis')



Build Spillover Graph

Resolve Minimum Z
Calculate Watershed Areas

Ensure epsilon Gradient : 0.000500 m
Raised 104 links
Saved to : ..\outputs\GLOBAL\DEM\FLAT_SPILLOVER_10K.npz
Build Spillover Graph

Resolve Minimum Z
Calculate Watershed Areas

Ensure epsilon Gradient : 0.000500 m
Raised 95 links
Saved to : ..\outputs\GLOBAL\DEM\FLAT_SPILLOVER_10KBIS.npz




Create virtual raster from tiles in shell (optional)

In [23]:
! fct-tiles -c ./config.ini buildvrt 10k dem-flat-labels
! fct-tiles -c ./config.ini buildvrt 10k dem-drainage-resolved

! fct-tiles -c ./config.ini buildvrt 10kbis dem-flat-labels
! fct-tiles -c ./config.ini buildvrt 10kbis dem-drainage-resolved

FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.


#### Flow direction

In [24]:
# import module
from fct.drainage import FlowDirection

In [25]:
# set parameters
params = FlowDirection.Parameters()

FlowDirection parameters :
- _exterior = exterior domain. not found in yml input, should be a shape file to indentify the area out of the DEM and avoid flow to go out of this mask (flow = -1). **utilisé?**
- _elevations = filled DEM tiles raster input from Resolve flats step.
- _flow = D8 Flow Direction output raster, derived from DEM

In [43]:
# uncomment below to show default parameters
# params.__dict__.items()

dict_items([('_exterior', 'exterior-domain'), ('_elevations', 'dem-drainage-resolved'), ('_flow', 'flow')])

In [26]:
# change default parameters
# deactivate this parameter, exterior shapefile not define
params.exterior = 'off'

In [27]:
# Calculate flow direction
FlowDirection.FlowDirection(params=params, overwrite=True, processes=4, tileset='10k')
FlowDirection.FlowDirection(params=params, overwrite=True, processes=4, tileset='10kbis')

Create virtual raster from tiles in shell (optional)

In [28]:
! fct-tiles -c ./config.ini buildvrt 10k flow

FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.


#### Flow accumulation

In [29]:
# import module
from fct.drainage import Accumulate

In [40]:
# set parameters
params = Accumulate.Parameters()

Flow accumulation parameters :
- _exterior_flow = shapefile to connect outlets flow. **Pas dans le yml ni dans config.ini...**
- _elevations = initial raster DEM input
- _flow = Flow Direction input raster from flow direction step
- _outlets = 

In [31]:
# uncomment below to show default parameters
params.__dict__.items()

dict_items([('_exterior_flow', 'off'), ('_elevations', 'dem'), ('_flow', 'flow'), ('_outlets', 'outlets'), ('_outlets_pattern', 'outlets-glob'), ('_inlets', 'inlets'), ('_inlet_areas', 'inlet-areas'), ('_acc', 'acc')])

In [32]:
# change default parameters
params.elevations = 'dem-drainage-resolved'

In [33]:
# Flow tiles inlets/outlets graph
Accumulate.Outlets(params=params, processes=4, tileset='10k')
Accumulate.Outlets(params=params, processes=4, tileset='10kbis')

Accumulate.AggregateOutlets(params, tileset='10k')
Accumulate.AggregateOutlets(params, tileset='10kbis')

# Resolve inlets/outlets graph
Accumulate.InletAreas(params=params, tileset='10k')
Accumulate.InletAreas(params=params, tileset='10kbis')

# Flow accumulation
Accumulate.FlowAccumulation(params=params, overwrite=True, processes=4, tileset='10k') 
Accumulate.FlowAccumulation(params=params, overwrite=True, processes=4, tileset='10kbis')

C:\Users\lmanie01\AppData\Local\Temp\ipykernel_12772\1505093030.py:5: FionaDeprecationWarning: This function will be removed in version 2.0. Please use CRS.from_epsg() instead.
  Accumulate.AggregateOutlets(params, tileset='10k')


C:\Users\lmanie01\AppData\Local\Temp\ipykernel_12772\1505093030.py:6: FionaDeprecationWarning: This function will be removed in version 2.0. Please use CRS.from_epsg() instead.
  Accumulate.AggregateOutlets(params, tileset='10kbis')


Build outlets graph

Created graph with 52196 nodes
Accumulate areas

Write inlet shapefiles

Build outlets graph

Created graph with 55850 nodes
Accumulate areas

Write inlet shapefiles





In [35]:
! fct-tiles -c ./config.ini buildvrt 10k dem-drainage-resolved
! fct-tiles -c ./config.ini buildvrt 10kbis dem-drainage-resolved

! fct-tiles -c ./config.ini buildvrt 10k acc
! fct-tiles -c ./config.ini buildvrt 10kbis acc

FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.
FCT_CONFIG=./config.ini
Read configuration from ./config.ini
Environment from C:\Users\lmanie01\Documents\Gitlab\python-fct\tutorials\01-drainage_plan\.env
0...10...20...30...40...50...60...70...80...90...100 - done.


#### Stream network from source

In [39]:
# import module
from fct.drainage import StreamSources

**Add parameters for StreamSources**

In [41]:
StreamSources.InletSources(params, tileset='10k')
StreamSources.InletSources(params, tileset='10kbis')

Build sources graph

Created graph with 52217 nodes
Accumulate areas

Write inlet shapefiles

Build sources graph

Created graph with 55871 nodes
Accumulate areas

Write inlet shapefiles



In [42]:
# uncomment below to see an example of the the output tile path
params.output.tilename(row=1, col=1)

AttributeError: 'Parameters' object has no attribute 'output'

All intermediate virtual raster